# The Patron
## An example of the Digital Twin

This example just includes:
* Connecting
* Uploading models
* Creating twins
* Querying twins

It's not a lot, but it will all tie together when we start running scenarios. 

[This is the SDK repo on Github](https://github.com/Azure/azure-sdk-for-python/tree/4559e19e2f3146a49f1eba1706bb798071f4a1f5/sdk/digitaltwins/azure-digitaltwins-core)

[Here is the doc on the query language](https://docs.microsoft.com/en-us/azure/digital-twins/concepts-query-language)


## Connecting

**note**: you will need to replace {`home-test-twin.api.wcus.digitaltwins.azure.net`} with the address of your ADT. 

In [2]:
from azure.identity import AzureCliCredential
from azure.digitaltwins.core import DigitalTwinsClient

# using yaml instead of 
import yaml
import uuid

# using altair instead of matplotlib for vizuals
import numpy as np
import pandas as pd

# you will get this from the ADT resource at portal.azure.com
your_digital_twin_url = "home-test-twin.api.wcus.digitaltwins.azure.net"

azure_cli = AzureCliCredential()
service_client = DigitalTwinsClient(
    your_digital_twin_url, azure_cli)
service_client

In [3]:
service_client

Loading the model from a local Json file, I'm defining a model for a `Patron`. That's like a `class` that will be used to make several instances of a customer.

## Uploading models

In [4]:
patron_model_id = "dtmi:mymodels:patron;1"

In [5]:
# # Delete the model that you don't want. 
# service_client.delete_model(patron_model_id)

# # Create it if you just deleted it.
# patron_model_json = yaml.safe_load(open("models/patron.json"))
# service_client.create_models([patron_model_json])

Note that the json must be in a list. This is so that you can deploy several models in a single go.

In [6]:
# Get the Patron model
get_model = service_client.get_model(patron_model_id)
get_model.as_dict()

{'display_name': {'en': 'Patron'},
 'description': {'en': 'As an example, contains all of the properties possible in the DTDL.'},
 'id': 'dtmi:billmanh:patron;1',
 'upload_time': '2020-11-19T02:14:27.21094Z',
 'decommissioned': False}

In my case I use `customer` as an _instance_ of a model (a _twin_). The only thing to add is the information specific to the twin I want to upload. 

## Creating twins

In [7]:
digital_twin_id = 'customer-' + str(uuid.uuid4())

customer_json = {
    "$metadata": {
        "$model": patron_model_id
    },
    "satisfaction": 10,
    "totalWaitTime": 10
}

created_twin = service_client.upsert_digital_twin(digital_twin_id, customer_json)

You can see that the process with the API is simple:
* Define a model
* Build a twin from that model
* Give that model some attributes.


In [7]:
type(created_twin)

dict

In [8]:
created_twin

{'$dtId': 'customer-3cbd5e60-957d-44ff-944f-9adb42a20a52',
 '$etag': 'W/"ea33ae65-f47c-4116-8e3d-1c6fa61777d6"',
 'satisfaction': 10,
 'totalWaitTime': 10,
 '$metadata': {'$model': 'dtmi:billmanh:patron;1',
  'satisfaction': {'lastUpdateTime': '2020-11-19T15:48:13.1057699Z'},
  'totalWaitTime': {'lastUpdateTime': '2020-11-19T15:48:13.1057699Z'}}}

So let's create a bunch of people. Let's pretend we have a single room full of a bunch of people that all have similar attributes:
* `satisfaction` varies from person to person.
* `totalWaitTime` is the same at the beginning. 

the item for the `$metadata` is the model that it will be an instance of. The rest of the items in the dict are the `contents` of the model as you defined it.


In [13]:
def create_new_customer():
    digital_twin_id = 'customer-' + str(uuid.uuid4())
    customer_json = {
        "$metadata": {
            "$model": patron_model_id
        },
        "satisfaction": np.random.randint(5,10),
        "totalWaitTime": 0
    }
    return digital_twin_id,customer_json
    
customer_twin_examples = [create_new_customer() for i in range(40)]
customer_twin_examples[:5]

[('customer-5c454e2f-f70b-4352-b75a-958f1a49beba',
  {'$metadata': {'$model': 'dtmi:billmanh:patron;1'},
   'satisfaction': 7,
   'totalWaitTime': 0}),
 ('customer-26196fee-5ffd-457a-86b7-192a998f3cf2',
  {'$metadata': {'$model': 'dtmi:billmanh:patron;1'},
   'satisfaction': 9,
   'totalWaitTime': 0}),
 ('customer-e6f49d8a-711b-41c3-9db8-c7ece3dbc32c',
  {'$metadata': {'$model': 'dtmi:billmanh:patron;1'},
   'satisfaction': 7,
   'totalWaitTime': 0}),
 ('customer-c87adbfa-1c6e-4ea9-9f03-83e3877ef5fc',
  {'$metadata': {'$model': 'dtmi:billmanh:patron;1'},
   'satisfaction': 8,
   'totalWaitTime': 0}),
 ('customer-21e17d28-76c3-4c04-8df9-396703692a68',
  {'$metadata': {'$model': 'dtmi:billmanh:patron;1'},
   'satisfaction': 8,
   'totalWaitTime': 0})]

In [22]:
customer_df = pd.DataFrame([i[1] for i in customer_twin_examples])

customer_df

alt.Chart(...)

Of course, those are just local customer data points. You could imagine that this is information collected at the edge. 

Now let's upload those people to see them in the cloud. Try to imagine that each customer in the store had an app on their phone that sent this information to the graph. Each `customer` represents a data feed from IoT telemetry. Generally, each device would upload a separate feed of IoT, but we are simulating using a single device (your computer) to upload this data.


In [24]:
for i in customer_twin_examples:
    service_client.upsert_digital_twin(i[0], i[1])

Now let's query our customers to see how they are feeling. 

## Querying twins
This is pretty easy. If you've ever done SQL you'll get this right away.


In [26]:
query_expression = 'SELECT * FROM digitaltwins'
query_result = service_client.query_twins(query_expression)

In [28]:
for i in query_result:
    pass

In [29]:
i

{'$dtId': 'customer-2e551c55-265a-46e0-a84b-4965bc734e21',
 '$etag': 'W/"8f1be988-70af-4488-a7c9-5403a4b6ea1d"',
 'satisfaction': 9,
 'totalWaitTime': 0,
 '$metadata': {'$model': 'dtmi:billmanh:patron;1',
  'satisfaction': {'lastUpdateTime': '2020-11-19T16:13:44.9882083Z'},
  'totalWaitTime': {'lastUpdateTime': '2020-11-19T16:13:44.9882083Z'}}}

Let's say I want to get all of the customers that have > 7 satisfaction _Happy customers_

In [15]:
query_result = service_client.query_twins(
"""
SELECT  *
FROM DigitalTwins T  
WHERE T.satisfaction > 7
"""
)

In [16]:
for i in query_result:
    print (i['$dtId'],i['satisfaction'])

customer-cc04f3b6-39b0-4cef-bfff-a7d668cce446 10
customer-3cbd5e60-957d-44ff-944f-9adb42a20a52 10
customer-26196fee-5ffd-457a-86b7-192a998f3cf2 9
customer-c87adbfa-1c6e-4ea9-9f03-83e3877ef5fc 8
customer-21e17d28-76c3-4c04-8df9-396703692a68 8
customer-8375cece-abf8-400f-be66-459f7f40fe6a 8
customer-d0483de8-0f9f-41d4-ac0c-84d608f7467a 8
customer-8ae03d4b-ee2c-4ffd-9aa8-8edbfbf39728 9
customer-fc568319-d5a9-492a-83ce-fed774257003 8
customer-d9ca8715-1a6d-43ea-83d2-671fc972ebd7 9
customer-acd7a91f-1d28-46b3-894d-6f3953fbd85e 8
customer-a867aacb-68a7-4872-a3bb-330b0b34ef7c 9
customer-373017d3-d7e2-4407-8c5b-5fc44e20286f 9
customer-418cbc74-b101-4f50-8cf0-6075a2a8053c 8
customer-04cc7f9d-5503-4d64-8dbc-373a47a8710a 9
customer-e5d090d6-ae68-4c31-890e-16ce96c1e462 9
customer-12a558cf-b15d-41ee-a98b-d07c4b7411ee 8
customer-1c51c2a0-45bc-442c-b4df-1d193e11f628 8
customer-ca674938-4d13-463f-8858-e0b9ac5deebc 9
customer-2139a9c9-ff79-4a63-9576-c34744f0521d 9
customer-2e551c55-265a-46e0-a84b-4965b

Of course, you can just pull everything, but this might be problematic as you build a larger model. 

In [33]:
query_result = service_client.query_twins(
"""
SELECT  COUNT() 
FROM DigitalTwins 
"""
)

In [34]:
for i in query_result:
    print(i)

{'COUNT': 88}
